# LSTM con Keras

Le reti **Long short-term memory (LSTM)** sono un'architettura di reti neurali ricorrenti che risolvono il problema della scomparsa del gradiente tra le diverse esecuzioni della rete.
<br>
Le LSTM  sono state da Sepp Hochreiter e Jurger Schmidhuber nel 1997 in [questa ricerca](https://www.bioinf.jku.at/publications/older/2604.pdf) ed in sostanza aggiungono uno stato alla rete (cell state) che viaggia in parallelo al segnale della rete e immagazzina le informazioni sequenziali.
<br><br>
In questo notebook utilizzeremo le LSTM per migliorare la rete neurale per la sentiment analysis addestrata sull'IMDB Movie Reviews dataset.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

## Scarichiamo il dataset

In [42]:
from keras.datasets import imdb 

num_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


## Preprocessiamo i dati

In [43]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 500

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 500)

## Creiamo il modello

In [46]:
from keras.layers import Embedding, LSTM

model = Sequential()

model.add(Embedding(num_words, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, None, 32)          320000    
_________________________________________________________________
lstm_21 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 40s 2ms/step - loss: 0.6260 - acc: 0.6710 - val_loss: 0.5237 - val_acc: 0.7602
Epoch 2/10
20000/20000 [==============================] - 36s 2ms/step - loss: 0.4206 - acc: 0.8332 - val_loss: 0.4284 - val_acc: 0.8258
Epoch 3/10
20000/20000 [==============================] - 36s 2ms/step - loss: 0.3251 - acc: 0.8751 - val_loss: 0.3541 - val_acc: 0.8578
Epoch 4/10
20000/20000 [==============================] - 35s 2ms/step - loss: 0.2820 - acc: 0.8918 - val_loss: 0.4675 - val_acc: 0.8090
Epoch 5/10
20000/20000 [==============================] - 35s 2ms/step - loss: 0.2445 - acc: 0.9087 - val_loss: 0.3054 - val_acc: 0.8786
Epoch 6/10
20000/20000 [==============================] - 35s 2ms/step - loss: 0.2193 - acc: 0.9197 - val_loss: 0.3926 - val_acc: 0.8628
Epoch 7/10
20000/20000 [==============================] - 36s 2ms/step - loss: 0.1915 - acc: 0.9311 - val_loss: 0.3110 - val_acc:

[0.66455034471035, 0.80752]

## Dropout in una RNN

In [32]:
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 32))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 100)         1000000   
_________________________________________________________________
lstm_17 (LSTM)               (None, 32)                17024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 1,017,057
Trainable params: 1,017,057
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, validation_split=0.2, epochs=20)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 22s 1ms/step - loss: 0.6156 - acc: 0.6881 - val_loss: 0.4987 - val_acc: 0.7988
Epoch 2/20
20000/20000 [==============================] - 17s 867us/step - loss: 0.4457 - acc: 0.8199 - val_loss: 0.3811 - val_acc: 0.8438
Epoch 3/20
20000/20000 [==============================] - 17s 859us/step - loss: 0.3710 - acc: 0.8519 - val_loss: 0.3742 - val_acc: 0.8388
Epoch 4/20
20000/20000 [==============================] - 17s 856us/step - loss: 0.3329 - acc: 0.8685 - val_loss: 0.3850 - val_acc: 0.8430
Epoch 5/20
20000/20000 [==============================] - 17s 850us/step - loss: 0.2952 - acc: 0.8876 - val_loss: 0.4362 - val_acc: 0.8250
Epoch 6/20
20000/20000 [==============================] - 17s 861us/step - loss: 0.2832 - acc: 0.8917 - val_loss: 0.3470 - val_acc: 0.8580
Epoch 7/20
20000/20000 [==============================] - 18s 876us/step - loss: 0.2584 - acc: 0.9022 - val_loss: 0.375

KeyboardInterrupt: ignored

## Aggiungiamo altri strati LSTM

In [25]:
from keras.layers import Embedding, LSTM

model = Sequential()

model.add(Embedding(num_words, 100))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(64, dropout=0.4, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 100)         1000000   
_________________________________________________________________
lstm_13 (LSTM)               (None, None, 32)          17024     
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 1,041,921
Trainable params: 1,041,921
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 85s 4ms/step - loss: 0.5919 - acc: 0.6885 - val_loss: 0.4701 - val_acc: 0.7732
Epoch 2/10
 1536/20000 [=>............................] - ETA: 1:13 - loss: 0.4208 - acc: 0.8190

KeyboardInterrupt: ignored